In [16]:
import geopandas as gpd

def assign_heights(source_geojson, target_geojson, output_geojson, height_attr='height',target_crs='EPSG:2056'):
    """
    Assign height values from source GeoJSON to target GeoJSON based on spatial proximity.

    :param source_geojson: Path to the GeoJSON file with height values.
    :param target_geojson: Path to the GeoJSON file without height values.
    :param output_geojson: Path to save the updated GeoJSON file.
    :param height_attr: The attribute name in the source GeoJSON containing height information.
    """
    # Load the GeoJSON files
    source_gdf = gpd.read_file(source_geojson)
    target_gdf = gpd.read_file(target_geojson)

    # Ensure both GeoDataFrames use the same CRS
    if source_gdf.crs != target_gdf.crs:
        target_gdf = target_gdf.to_crs(source_gdf.crs)
    
    # Ensure unique indices for both GeoDataFrames before performing the spatial join
    source_gdf = source_gdf.reset_index(drop=True)
    target_gdf = target_gdf.reset_index(drop=True)

    # Perform a spatial join to match target geometries with the closest source geometries
    joined_gdf = gpd.sjoin_nearest(target_gdf, source_gdf[[height_attr, 'geometry']], how='left', distance_col='distance')

    # Drop duplicate rows based on 'index_right' (the index of the matched source geometries)
    # This ensures each row in the target GeoDataFrame gets only one height value.
    joined_gdf = joined_gdf.drop_duplicates(subset='index_right', keep='first')

    # Reset index of joined_gdf to align with target_gdf
    joined_gdf = joined_gdf.reset_index(drop=True)

    # Transfer height values to target GeoDataFrame
    target_gdf[height_attr] = joined_gdf[height_attr]
    
    # Set height to 0 for all non-NaN height values
    target_gdf[height_attr] = target_gdf[height_attr].fillna(0)
    
    # Explicitly set the CRS for the target GeoDataFrame before saving
    target_gdf = target_gdf.set_crs(target_crs, allow_override=True)

    # Save the updated GeoDataFrame to a new GeoJSON file
    target_gdf.to_file(output_geojson, driver='GeoJSON')
    
    print(f"Height values assigned and saved to {output_geojson}")


In [17]:
base_path = "/Volumes/T7 Shield/GMP_Data/processed_data/00_Segmentation/"

source_geojson = base_path + '1975/skeleton_1975.geojson'
target_geojson = base_path + '1899/skeleton_1899.geojson'

output_geojson = base_path + '1899/skeleton_1899_heights.geojson'

assign_heights(source_geojson=source_geojson, target_geojson=target_geojson, output_geojson=output_geojson)


/Users/mischabauckhage/opt/anaconda3/envs/gis/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


Height values assigned and saved to /Volumes/T7 Shield/GMP_Data/processed_data/00_Segmentation/1899/skeleton_1899_heights.geojson


In [4]:
source_gdf = gpd.read_file(source_geojson)
target_gdf = gpd.read_file(target_geojson)

In [5]:
source_gdf.head()

,color,height,geometry
0,"rgb(34.7259649814772, 33.37307021087659, 70.52...",520,"LINESTRING (2.5822e+06 1.1952e+06, 2.5822e+06 ..."
1,"rgb(122.18593199046393, 175.14270568183994, 14...",540,"LINESTRING (2.5817e+06 1.1949e+06, 2.5817e+06 ..."
2,"rgb(160.68566675478058, 54.98538402197648, 163...",550,"LINESTRING (2.5819e+06 1.194e+06, 2.5819e+06 1..."
3,"rgb(125.15551947581321, 122.69872785673837, 13...",550,"LINESTRING (2.5816e+06 1.194e+06, 2.5816e+06 1..."
4,"rgb(191.09050775321427, 169.14806939538468, 11...",560,"LINESTRING (2.5816e+06 1.1938e+06, 2.5816e+06 ..."


In [6]:
target_gdf.head()

,color,geometry
0,"rgb(41.19139747571343, 76.52709651869034, 6.67...","LINESTRING (2.5816e+06 1.1953e+06, 2.5816e+06 ..."
1,"rgb(190.87333485768073, 61.04649349667606, 80....","LINESTRING (2.5826e+06 1.1954e+06, 2.5826e+06 ..."
2,"rgb(197.01536947936776, 101.05657516967675, 3....","LINESTRING (2.583e+06 1.1954e+06, 2.583e+06 1...."
3,"rgb(123.84123779613385, 72.72195377483608, 99....","LINESTRING (2.5821e+06 1.1952e+06, 2.5821e+06 ..."
4,"rgb(146.023092235117, 60.64756080877301, 159.4...","LINESTRING (2.5816e+06 1.195e+06, 2.5816e+06 1..."
